In [4]:
import numpy as np
import pandas as pd
import nltk
import string
#import fasttext
#import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

In [2]:
## Process Data
rawData = pd.read_csv('potential-talents - Aspiring human resources - seeking human resources.csv')

In [24]:
# Copy and merge in the location data
textProcessing = rawData.copy()
textProcessing['mergeColumns'] = textProcessing['job_title'].astype(str) + " " + \
                                 textProcessing['location'].astype(str)
    
# Tokenize merged column
textProcessing['tokenized'] = textProcessing['mergeColumns'].apply(word_tokenize)

# Move to lowercase, remove punctuation
textProcessing['lower'] = textProcessing['tokenized'].apply(lambda x: [word.lower() for word in x])
textProcessing['no_punc'] = textProcessing['lower'].apply(lambda x: [word for word in x if word not in string.punctuation])

# Remove stop words
stop_words = set(stopwords.words('english'))
stop_words.add('area')
stop_words.add('greater')
textProcessing['stopwords_removed'] = textProcessing['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
textProcessing['jobTitleString'] = [' '.join(map(str,l)) for l in textProcessing['stopwords_removed']]

# Convert connections
textProcessing['connectionsInt'] = textProcessing['connection'].str.replace(r'\D', '').astype(int)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: The default value of regex will change from True to False in a future version.


In [25]:
textProcessing

,id,job_title,location,connection,fit,mergeColumns,tokenized,lower,no_punc,stopwords_removed,jobTitleString,connectionsInt
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN,2019 C.T. Bauer College of Business Graduate (...,"[2019, C.T, ., Bauer, College, of, Business, G...","[2019, c.t, ., bauer, college, of, business, g...","[2019, c.t, bauer, college, of, business, grad...","[2019, c.t, bauer, college, business, graduate...",2019 c.t bauer college business graduate magna...,85
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN,Native English Teacher at EPIK (English Progra...,"[Native, English, Teacher, at, EPIK, (, Englis...","[native, english, teacher, at, epik, (, englis...","[native, english, teacher, at, epik, english, ...","[native, english, teacher, epik, english, prog...",native english teacher epik english program ko...,500
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional Raleigh-...,"[Aspiring, Human, Resources, Professional, Ral...","[aspiring, human, resources, professional, ral...","[aspiring, human, resources, professional, ral...","[aspiring, human, resources, professional, ral...",aspiring human resources professional raleigh-...,44
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,"People Development Coordinator at Ryan Denton,...","[People, Development, Coordinator, at, Ryan, D...","[people, development, coordinator, at, ryan, d...","[people, development, coordinator, at, ryan, d...","[people, development, coordinator, ryan, dento...",people development coordinator ryan denton texas,500
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,Advisory Board Member at Celal Bayar Universit...,"[Advisory, Board, Member, at, Celal, Bayar, Un...","[advisory, board, member, at, celal, bayar, un...","[advisory, board, member, at, celal, bayar, un...","[advisory, board, member, celal, bayar, univer...",advisory board member celal bayar university i...,500
...,...,...,...,...,...,...,...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,NaN,Aspiring Human Resources Manager | Graduating ...,"[Aspiring, Human, Resources, Manager, |, Gradu...","[aspiring, human, resources, manager, |, gradu...","[aspiring, human, resources, manager, graduati...","[aspiring, human, resources, manager, graduati...",aspiring human resources manager graduating ma...,103
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,NaN,Human Resources Generalist at Loparex Raleigh-...,"[Human, Resources, Generalist, at, Loparex, Ra...","[human, resources, generalist, at, loparex, ra...","[human, resources, generalist, at, loparex, ra...","[human, resources, generalist, loparex, raleig...",human resources generalist loparex raleigh-dur...,500
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN,Business Intelligence and Analytics at Travele...,"[Business, Intelligence, and, Analytics, at, T...","[business, intelligence, and, analytics, at, t...","[business, intelligence, and, analytics, at, t...","[business, intelligence, analytics, travelers,...",business intelligence analytics travelers new ...,49
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN,Always set them up for Success Greater Los Ang...,"[Always, set, them, up, for, Success, Greater,...","[always, set, them, up, for, success, greater,...","[always, set, them, up, for, success, greater,...","[always, set, success, los, angeles]",always set success los angeles,500


In [71]:
def textProcessing(data=None):
    data['mergeColumns'] = data['job_title'].astype(str) #+ " " + data['location'].astype(str)

    # Tokenize merged column
    data['tokenized'] = data['mergeColumns'].apply(word_tokenize)

    # Move to lowercase, remove punctuation
    data['lower'] = data['tokenized'].apply(lambda x: [word.lower() for word in x])
    data['no_punc'] = data['lower'].apply(lambda x: [word for word in x if word not in string.punctuation])

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    stop_words.add('area')
    stop_words.add('greater')
    data['stopwords_removed'] = data['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
    data['jobTitleString'] = [' '.join(map(str,l)) for l in data['stopwords_removed']]

    # Convert connections
    data['connectionsInt'] = data['connection'].str.replace(r'\D', '').astype(int)  
    
    return data

def getTFIDF(data=None, searchTermSize=1):
    vectorizer = TfidfVectorizer(max_df=0.9, min_df=3, max_features=100, use_idf=True, ngram_range=(searchTermSize, searchTermSize))
    tfidf = vectorizer.fit_transform(data)
    tfidfNames = vectorizer.get_feature_names()
    tfidfDataFrame = pd.DataFrame(tfidf.toarray(), columns=list(tfidfNames))
    return tfidfDataFrame

def genRankings(data=None, searchTerm='student'):
    tfidf = getTFIDF(data=data['jobTitleString'], searchTermSize=len(searchTerm.split(' ')))
    mergedDataFrames = pd.concat([data, tfidf], axis=1)
    mergedDataFrames['fit'] = mergedDataFrames[searchTerm] + mergedDataFrames[searchTerm]*mergedDataFrames['connectionsInt']/500.0
    return mergedDataFrames

def starred(fullData=None, rankedData=None, starredEntry=4):
    newSearchTerm = fullData.loc[fullData.id == rankedData.iloc[starredEntry].id].jobTitleString.values[0]
    print (newSearchTerm)
    newMerged = genRankings(data=fullData.copy(), searchTerm=newSearchTerm)
    newRanked = newMerged[['id', 'job_title', 'location', 'connection', 'fit']].copy()
    newRanked['fit'] = newRanked['fit']/newRanked['fit'].max()
    newRanked = newRanked.sort_values(by='fit', ascending=False)
    return newRanked
    
merged = genRankings(data=textProcessing(rawData.copy()), searchTerm = 'aspiring human resources')
ranked = merged[['id', 'job_title', 'location', 'connection', 'fit']].copy()
ranked['fit'] = ranked['fit']/ranked['fit'].max()
ranked = ranked.sort_values(by='fit', ascending=False)
ranked.reset_index(drop=True)[0:12]

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: The default value of regex will change from True to False in a future version.


,id,job_title,location,connection,fit
0,79,Liberal Arts Major. Aspiring Human Resources A...,"Baton Rouge, Louisiana Area",7,1.000000
1,76,Aspiring Human Resources Professional | Passio...,"New York, New York",212,0.862942
2,29,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,0.860928
3,27,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,0.860928
4,82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,0.816886
5,97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,0.692050
6,66,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57,0.675082
7,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.659326
8,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.659326
9,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.659326


In [73]:
starred(fullData=merged, rankedData=ranked, starredEntry=5)[0:12]

aspiring human resources professional


,id,job_title,location,connection,fit
75,76,Aspiring Human Resources Professional | Passio...,"New York, New York",212,1.000000
81,82,Aspiring Human Resources Professional | An ene...,"Austin, Texas Area",174,0.946629
96,97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,0.801966
65,66,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57,0.782303
45,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.764045
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.764045
20,21,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.764045
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.764045
57,58,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.764045
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.764045
